In [1]:
import torch
import pandas as pd
import torch_geometric
from torch_geometric.data import Dataset, Data
import numpy as np 
import os
from tqdm import tqdm
import torch.nn.functional as F 
from torch.nn import Linear, BatchNorm1d, ModuleList
from torch_geometric.nn import TransformerConv, TopKPooling 
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import mlflow.pytorch
from torch_geometric.data import DataLoader
import torch
import torch.nn.functional as F 
from torch.nn import Sequential, Linear, BatchNorm1d, ReLU
from torch_geometric.nn import TransformerConv, GATConv, TopKPooling, BatchNorm
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.nn.conv.x_conv import XConv
import torch 
from torch_geometric.data import DataLoader
from sklearn.metrics import confusion_matrix, f1_score, \
    accuracy_score, precision_score, recall_score, roc_auc_score
import numpy as np
from tqdm import tqdm
import mlflow.pytorch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
print(f"Torch version: {torch.__version__}")
print(f"Cuda available: {torch.cuda.is_available()}")
print(f"Torch geometric version: {torch_geometric.__version__}")

Torch version: 1.11.0
Cuda available: True
Torch geometric version: 2.0.5.dev20220428


In [3]:
import pickle
path_temp= "dataset-call-graph-blogpost-material/dataset/"
good_data=pickle.load(open(path_temp+"goodware_graphs.p","rb"))
bad_data=pickle.load(open(path_temp+"malware_graphs.p","rb"))

# Dataset

In [4]:

class MoleculeDataset(Dataset):
    def __init__(self, root, filename,good_data, bad_data, test=False, transform=None, pre_transform=None):
        """
        root = Where the dataset should be stored. This folder is split
        into raw_dir (downloaded dataset) and processed_dir (processed data). 
        """
        self.test = test
        self.filename = filename
        self.good_data=good_data
        self.bad_data=bad_data
        super(MoleculeDataset, self).__init__(root, transform, pre_transform)
        self.data_passed=self.good_data+self.bad_data
        
    @property
    def raw_file_names(self):
        """ If this file exists in raw_dir, the download is not triggered.
            (The download func. is not implemented here)  
        """
        return ["goodware_graphs.p","malware_graphs.p"]

    @property
    def processed_file_names(self):
        """ If these files are found in raw_dir, processing is skipped"""
#         self.data = self.data_passed
        return "NOt implemented yet"

        # if self.test:
        #     return [f'data_test_{i}.pt' for i in list(self.data.index)]
        # else:
        #     return [f'data_{i}.pt' for i in list(self.data.index)]

    def download(self):
        pass

    def process(self):
#         self.data = self.data_passed
        print("started doing stuff")
        index=0

        for mol in (self.good_data):
            

            feature_data=mol[0]
            neighbour_data=mol[1]

            # i need to create inst ->index
            instr_index=dict()
            for a,b in enumerate(feature_data):
              instr_index[b]=a 

            # Get node features
            node_feats = self._get_node_features(feature_data,instr_index)
            # Get edge features
            edge_feats = self._get_edge_features(neighbour_data)
#             edge_feats = []
            # Get adjacency info
            edge_index = self._get_adjacency_info(neighbour_data,instr_index)

            # Get labels info
            label = 1

            # Create data object
            data = Data(x=node_feats, 
                        edge_index=edge_index,
                        edge_attr=edge_feats,
                        y=label,
                        smiles=mol
                        ) 
                        
            torch.save(data, 
                    os.path.join(self.processed_dir, 
                                 f'data_{index}.pt'))

            index+=1
        
        for mol in (self.bad_data):
            

            feature_data=mol[0]
            neighbour_data=mol[1]

            # i need to create inst ->index
            instr_index=dict()
            for a,b in enumerate(feature_data):
              instr_index[b]=a 

            # Get node features
            node_feats = self._get_node_features(feature_data,instr_index)
            # Get edge features
            edge_feats = self._get_edge_features(neighbour_data)
#             edge_feats = []
            # Get adjacency info
            edge_index = self._get_adjacency_info(neighbour_data,instr_index)

            # Get labels info
            label = 0

            # Create data object
            data = Data(x=node_feats, 
                        edge_index=edge_index,
                        edge_attr=edge_feats,
                        y=label,
                        smiles=mol
                        ) 
                        
            torch.save(data, 
                    os.path.join(self.processed_dir, 
                                 f'data_{index}.pt'))

            index+=1
        
        

    def process_node_features(self,val, node_feature_menu):
      # print("jj")
      # print(val)
      node_out=[]
      keys=val.keys()
      for name in node_feature_menu : 
        if(name in keys):
          node_out.append(val[name])
        else:
          node_out.append(0)
      return node_out


    def _get_node_features(self, node_features,instr_index):
        """ 
        This will return a matrix / 2d array of the shape
        [Number of Nodes, Node Feature size]
        """
        node_feature_menu=['mov', 'call', 'lea', 'jmp', 'push', 'add', 'xor', 'cmp', 'int3', 'nop', 'pushl', 'dec', 'sub', 'insl', 'inc','jz', 'jnz', 'je', 'jne', 'ja', 'jna', 'js', 'jns', 'jl', 'jnl', 'jg', 'jng']

        all_node_feats = []

        for atom in node_features:
            # print("atom is",atom)
            
            node_feats = []
            node_feats = self.process_node_features(node_features[atom],node_feature_menu)
            # Append node features to matrix
            all_node_feats.append(node_feats)

        all_node_feats = np.asarray(all_node_feats)
        # all_node_feats = np.transpose(all_node_feats)
        return torch.tensor(all_node_feats, dtype=torch.float)

    def _get_edge_features(self, mol):
        """ 
        This will return a matrix / 2d array of the shape
        [Number of edges, Edge Feature size]
        """
        all_edge_feats = []

        # for bond in mol.GetBonds():
        #     edge_feats = []
        #     # Feature 1: Bond type (as double)
        #     edge_feats.append(bond.GetBondTypeAsDouble())
        #     # Feature 2: Rings
        #     edge_feats.append(bond.IsInRing())
        #     # Append node features to matrix (twice, per direction)
        #     all_edge_feats += [edge_feats, edge_feats]

        all_edge_feats = np.asarray(all_edge_feats)
        return torch.tensor(all_edge_feats, dtype=torch.float)

    def get_one_instr_adjacency(self,val,instr_index,self_instruction_number):
        edge_val=[]
        for a in val:
            edge_val+=[[instr_index[self_instruction_number],instr_index[a]]]

        return edge_val

    def _get_adjacency_info(self, mol, instr_index):
        """
        We could also use rdmolops.GetAdjacencyMatrix(mol)
        but we want to be sure that the order of the indices
        matches the order of the edge features
        """
        edge_indices = []
        for bond in mol:
            edge_indices += self.get_one_instr_adjacency(mol[bond],instr_index,bond)

        # print("--------")
        # print(len(edge_indices))
        # print(len(edge_indices[0]))
        # print(len(edge_indices[1]))
        # print(edge_indices)
        # edge_indices = torch.tensor(edge_indices)
        # edge_indices = edge_indices.t().to(torch.long).view(2, -1)

        np_edge=np.array(edge_indices)
        np_edge=np.transpose(np_edge)
        tr_edge=torch.tensor(np_edge, dtype=torch.long)


        return tr_edge

    def _get_labels(self, label):
        label = np.asarray([label])
        return torch.tensor(label, dtype=torch.int64)

    def len(self):
#         return self.data_pass.shape[0]
        return len(self.data_passed)

    def get(self, idx):
        """ - Equivalent to __getitem__ in pytorch
            - Is not needed for PyG's InMemoryDataset
        """
        data = torch.load(os.path.join(self.processed_dir, 
                             f'data_{idx}.pt'))
        return data





In [5]:
# train_dataset = MoleculeDataset(root="data/", filename="goodware_graphs.p", good_data=good_data[:100], bad_data=bad_data[:100])
train_dataset = MoleculeDataset(root="data/", filename="goodware_graphs.p", good_data=good_data, bad_data=bad_data)

Processing...


started doing stuff


Done!


In [6]:
# test_cut_dataset=train_dataset[526:566]
# train_cut_dataset= train_dataset[:526]+train_dataset[566:]


NUM_GRAPHS_PER_BATCH = 1

train_loader = DataLoader(train_dataset, 
                    batch_size=NUM_GRAPHS_PER_BATCH,
                     shuffle=False,
                         num_workers=0)
test_loader = DataLoader(train_dataset, 
                    batch_size=NUM_GRAPHS_PER_BATCH,
                     shuffle=False)


d:\Anaconda3\envs\pytorch\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


# Model

In [7]:
class GNN(torch.nn.Module):
    def __init__(self, feature_size):
        super(GNN, self).__init__()
        num_classes = 2
        embedding_size = 128

        # GNN layers
        self.conv1 = GATConv(feature_size, embedding_size, heads=3, dropout=0.3)
        self.conv2 = GATConv(embedding_size*3, embedding_size, heads=3, dropout=0.3)
        self.conv3 = GATConv(embedding_size*3,2, heads=1, dropout=0.3)
        # self.readout = GlobalMeanPool()

    def forward(self, x, edge_attr, edge_index, batch):
        # x, edge_index, batch = self.arguments_read(*args, **kwargs)
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.conv3(x, edge_index)
        # x = x.mean(dim=1)  # 464
        x = x.mean(dim=0)  # 2
        
        return x


In [8]:
import torch.nn as nn

class GNNPool(nn.Module):
    def __init__(self):
        super().__init__()


class GlobalMeanPool(GNNPool):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return gnn.global_mean_pool(x, batch)


In [9]:


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [10]:
#%% Loading the model
# model = GNN(feature_size=train_dataset[0].x.shape[0]) 
model = GNN(feature_size=train_dataset[0].x.shape[1]) 
model = model.to(device)
print(f"Number of parameters: {count_parameters(model)}")
weights = torch.tensor([1, 1], dtype=torch.float32).to(device)
loss_fn = torch.nn.CrossEntropyLoss(weight=weights)
optimizer = torch.optim.Adam(model.parameters(), lr=0.000001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.2, amsgrad=False)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.008)
model

Number of parameters: 160902


GNN(
  (conv1): GATConv(27, 128, heads=3)
  (conv2): GATConv(384, 128, heads=3)
  (conv3): GATConv(384, 2, heads=1)
)

# train and test

In [11]:
def test(epoch):
    all_preds = []
    all_labels = []
    for batch in test_loader:
        batch.to(device)  
        pred = model(batch.x.float(), 
                        batch.edge_attr.float(),
                        batch.edge_index, 
                        batch.batch
                        ) 
        loss = torch.sqrt(loss_fn(pred, batch.y))    
        all_preds.append(np.argmax(pred.cpu().detach().numpy(), axis=1))
        all_labels.append(batch.y.cpu().detach().numpy())

    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    calculate_metrics(all_preds, all_labels, epoch, "test")
    return loss


def calculate_metrics(y_pred, y_true, epoch, type):
    print(f"\n Confusion matrix: \n {confusion_matrix(y_pred, y_true)}")
    print(f"F1 Score: {f1_score(y_pred, y_true)}")
    print(f"Accuracy: {accuracy_score(y_pred, y_true)}")
    print(f"Precision: {precision_score(y_pred, y_true)}")
    print(f"Recall: {recall_score(y_pred, y_true)}")
    try:
        roc = roc_auc_score(y_pred, y_true)
        print(f"ROC AUC: {roc}")
        mlflow.log_metric(key=f"ROC-AUC-{type}", value=float(roc), step=epoch)
    except:
        mlflow.log_metric(key=f"ROC-AUC-{type}", value=float(0), step=epoch)
        print(f"ROC AUC: notdefined")
        
    try:
        mlflow.log_metric(key=f"accuracy-{type}", value=float(accuracy_score(y_pred, y_true)), step=epoch)
    except:
        mlflow.log_metric(key=f"ROC-AUC-{type}", value=float(0), step=epoch)
        print(f"Accuracy: notdefined")

In [15]:

def train(epoch):
    # Enumerate over the data
    all_preds = []
    all_labels = []
    for _, batch in enumerate(tqdm(train_loader)):
        # Use GPU
        # print("--------------------------------------------------")
        batch.to(device)  
        # Reset gradients

        optimizer.zero_grad() 
        # Passing the node features and the connection info
        pred = model(batch.x.float(), 
                                batch.edge_attr.float(),
                                batch.edge_index, 
                                batch.batch
                                ) 
        pred=pred.resize(1,pred.size()[0])
        # Calculating the loss and gradients
        #  loss = torch.sqrt(loss_fn(pred, batch.y))
 
        # pred=pred.resize(1,pred.size()[0])
        sfm = nn.Softmax(dim=1)
        pred=sfm(pred) 
        
        # print("predictions",pred)

        # pred=pred.data.argmax()
        # print("predictions",pred.size())
        # print("predictions",pred.size())


        loss_v1 =  loss_fn(pred, batch.y) 
        # print("loss_v1",loss_v1)
        loss_v2=torch.sqrt(loss_v1)
        # print("loss_v2",loss_v2)
        loss=loss_v2
        
        loss.backward()  

        
        # this is the additional clipping
        torch.nn.utils.clip_grad_value_(model.parameters(), clip_value=7.0)
        
        
        # Update using the gradients
        optimizer.step()  


        all_preds.append(np.argmax(pred.cpu().detach().numpy(), axis=1))
        all_labels.append(batch.y.cpu().detach().numpy())
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    calculate_metrics(all_preds, all_labels, epoch, "train")
    return loss




# lets train

In [16]:
from torch_geometric.nn import TransformerConv, GATConv, TopKPooling, BatchNorm


In [19]:

# %% Run the training
with mlflow.start_run() as run:
    for epoch in range(20):
        # Training
        model.train()
        loss = train(epoch=epoch)
        # loss = loss.detach().gpu().numpy()
        loss = loss.detach().numpy()
        print(f"Epoch {epoch} | Train Loss {loss}")
        mlflow.log_metric(key="Train loss", value=float(loss), step=epoch)

        # Testing
        model.eval()
        if epoch % 10 == 0 and epoch !=0:
            loss = test(epoch=epoch)
            # loss = loss.detach().gpu().numpy()
            loss = loss.detach().numpy()
            print(f"Epoch {epoch} | Test Loss {loss}")
            mlflow.log_metric(key="Test loss", value=float(loss), step=epoch)

        scheduler.step()
print("Done.")



  0%|          | 0/1361 [00:00<?, ?it/s]d:\Anaconda3\envs\pytorch\lib\site-packages\torch\_tensor.py:549: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
100%|██████████| 1361/1361 [00:40<00:00, 33.68it/s]



 Confusion matrix: 
 [[673 373]
 [142 173]]
F1 Score: 0.40185830429732866
Accuracy: 0.6216017634092579
Precision: 0.31684981684981683
Recall: 0.5492063492063493
ROC AUC: 0.5963048954444748


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:
train_dataset[0].y

1

In [ ]:
aa=torch.randn(4)
print(aa)

tensor([-0.3508,  1.6014, -1.8914, -0.4558])


In [ ]:
bb=torch.ones(1)
bb=aa.resize(1,aa.size()[0])
print(bb)

tensor([[-0.3508,  1.6014, -1.8914, -0.4558]])


d:\Anaconda3\envs\pytorch\lib\site-packages\torch\_tensor.py:549: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


In [ ]:
m = nn.Softmax(dim=1)
print(m)
input = torch.randn(1, 3)
print(input)
output = m(input)
print(output)
print(input.size())
print(output.size())

Softmax(dim=1)
tensor([[ 0.1413, -0.3490,  0.5768]])
tensor([[0.3166, 0.1939, 0.4895]])
torch.Size([1, 3])
torch.Size([1, 3])


In [158]:

asss=torch.nn.CrossEntropyLoss(aa,bb)


RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [94]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(1, 5, requires_grad=True)
target = torch.empty(1, dtype=torch.long).random_(5)
output = loss(input, target)
output.backward()

# Example of target with class probabilities
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.randn(3, 5).softmax(dim=1)
print(input)
print(target)
# output = loss(input, target)
# output.backward()

tensor([[-0.0854, -0.3828, -0.8074,  0.5448,  1.0877]], requires_grad=True)
tensor([1])


In [ ]:
# # for i in range(len(train_dataset)):
#     # if (len(train_dataset[i].x[1])) == 768:
#         # print(i)

# for i in range(20):
#     print(train_dataset[i].x.size(),train_dataset[i].edge_index.size())


torch.Size([464, 27]) torch.Size([2, 821])
torch.Size([492, 27]) torch.Size([2, 1217])
torch.Size([485, 27]) torch.Size([2, 925])
torch.Size([440, 27]) torch.Size([2, 792])
torch.Size([465, 27]) torch.Size([2, 836])
torch.Size([468, 27]) torch.Size([2, 855])
torch.Size([440, 27]) torch.Size([2, 792])
torch.Size([587, 27]) torch.Size([2, 1083])
torch.Size([1137, 27]) torch.Size([2, 3479])
torch.Size([478, 27]) torch.Size([2, 866])
torch.Size([1024, 27]) torch.Size([2, 2837])
torch.Size([481, 27]) torch.Size([2, 690])
torch.Size([488, 27]) torch.Size([2, 928])
torch.Size([322, 27]) torch.Size([2, 354])
torch.Size([1122, 27]) torch.Size([2, 2241])
torch.Size([559, 27]) torch.Size([2, 982])
torch.Size([1643, 27]) torch.Size([2, 4396])
torch.Size([568, 27]) torch.Size([2, 1731])
torch.Size([463, 27]) torch.Size([2, 816])
torch.Size([484, 27]) torch.Size([2, 925])


In [ ]:
from datetime import date
from datetime import datetime

# %% Save the model 
model_type_name="20epoch_gpu"

mlflow.pytorch.log_model(model, "model") 
mlflow.end_run()


# SAVE MODEL
#  pls create a folder named model_saved if not there
now = datetime.now()
dt_string = now.strftime("%d_%m_%Y %H_%M_%S")
PATH_to_save_model="model_saved/"+"model_"+dt_string+":"+model_type_name+".pt"
PATH_to_save_model="model_saved/model "+dt_string+model_type_name+".pt"
torch.save(model, PATH_to_save_model)

2022/04/28 23:57:54 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\JERRYJ~1\AppData\Local\Temp\tmpajrx1zfm\model\data, flavor: pytorch), fall back to return ['torch==1.11.0', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback.
D:\Anaconda3\envs\pytorch\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [ ]:
PATH_to_save_model

'model_saved/model 28_04_2022 23_57_541epoch_cpu.pt'

In [69]:
# PATH_to_save_model='model_saved/model 25_04_2022 13_52_341epoch_ennexp.pt'
# PATH_to_save_model='model_saved/model 25_04_2022 14_50_061epoch_ennexp.pt'

In [18]:
PATH_TO_LOAD="model_saved/model 28_04_2022 23_57_541epoch_cpu.pt"

In [19]:
model=torch.load(PATH_TO_LOAD)
model.eval()

GNN(
  (conv1): GATConv(27, 256, heads=3)
  (head_transform1): Linear(in_features=768, out_features=256, bias=True)
  (pool1): TopKPooling(256, ratio=0.8, multiplier=1.0)
  (conv2): GATConv(256, 256, heads=3)
  (head_transform2): Linear(in_features=768, out_features=256, bias=True)
  (pool2): TopKPooling(256, ratio=0.5, multiplier=1.0)
  (conv3): GATConv(256, 256, heads=3)
  (head_transform3): Linear(in_features=768, out_features=256, bias=True)
  (pool3): TopKPooling(256, ratio=0.2, multiplier=1.0)
  (linear1): Linear(in_features=512, out_features=1024, bias=True)
  (linear2): Linear(in_features=1024, out_features=512, bias=True)
  (linear3): Linear(in_features=512, out_features=128, bias=True)
  (linear4): Linear(in_features=128, out_features=2, bias=True)
)

# XAI


In [20]:
from dig.xgraph.method import SubgraphX
explainer = SubgraphX(model, num_classes=4, device=device, explain_graph=False,
                        reward_method='nc_mc_l_shapley')

ImportError: DLL load failed while importing rdBase: The specified module could not be found.

In [ ]:
from dig.xgraph.method.subgraphx import find_closest_node_result
plotutils = PlotUtils(dataset_name='ba_shapes')

# Visualization
max_nodes = 5
node_idx = node_indices[20]
print(f'explain graph node {node_idx}')
data.to(device)
logits = model(data.x, data.edge_index)
prediction = logits[node_idx].argmax(-1).item()

_, explanation_results, related_preds = \
    explainer(data.x, data.edge_index, node_idx=node_idx, max_nodes=max_nodes)
result = find_closest_node_result(explanation_results[prediction], max_nodes=max_nodes)

plotutils = PlotUtils(dataset_name='ba_shapes')
explainer.visualization(explanation_results,
                        prediction,
                        max_nodes=max_nodes,
                        plot_utils=plotutils,
                        y=data.y)

In [47]:
from torch_geometric.nn import GNNExplainer
# Initialize explainer
explainer = GNNExplainer(model, epochs=200, return_type='log_prob')
graph = train_dataset[1]

In [48]:
graph.edge_index.size()

torch.Size([2, 1217])

In [55]:
torch.zeros(0)

tensor([])

In [56]:
# node_feat_mask, edge_mask = explainer.explain_graph(graph.x, graph.edge_index, batch_index=1)
# node_feat_mask, edge_mask = explainer.explain_graph(graph.x, graph.edge_index, batch=3)
# node_feat_mask, edge_mask = explainaer.explain_graph(graph.x, graph.edge_index, edge_attr=2)
node_feat_mask, edge_mask = explainer.explain_graph(graph.x, graph.edge_index, edge_attr=torch.zeros(0))

TypeError: forward() got multiple values for argument 'edge_attr'

In [ ]:
# for i in range(len(train_dataset)):
for i in range(20):
    print(train_dataset[i].x.size(),train_dataset[i].edge_index.size())

torch.Size([27, 464]) torch.Size([2, 821])
torch.Size([27, 492]) torch.Size([2, 1217])
torch.Size([27, 485]) torch.Size([2, 925])
torch.Size([27, 440]) torch.Size([2, 792])
torch.Size([27, 465]) torch.Size([2, 836])
torch.Size([27, 468]) torch.Size([2, 855])
torch.Size([27, 440]) torch.Size([2, 792])
torch.Size([27, 587]) torch.Size([2, 1083])
torch.Size([27, 1137]) torch.Size([2, 3479])
torch.Size([27, 478]) torch.Size([2, 866])
torch.Size([27, 1024]) torch.Size([2, 2837])
torch.Size([27, 481]) torch.Size([2, 690])
torch.Size([27, 488]) torch.Size([2, 928])
torch.Size([27, 322]) torch.Size([2, 354])
torch.Size([27, 1122]) torch.Size([2, 2241])
torch.Size([27, 559]) torch.Size([2, 982])
torch.Size([27, 1643]) torch.Size([2, 4396])
torch.Size([27, 568]) torch.Size([2, 1731])
torch.Size([27, 463]) torch.Size([2, 816])
torch.Size([27, 484]) torch.Size([2, 925])


In [ ]:
train_dataset[i].edge_index

tensor([[  0,   0,   0,  ..., 559, 560, 561],
        [531, 546, 254,  ..., 120, 522, 523]])

In [ ]:
# train_dataset[i].x
# len(train_dataset[i].x)
# train_dataset[0].x.size()
train_dataset[0].x.size()


torch.Size([464, 27])

In [ ]:
train_dataset[i].edge_attr

tensor([])

In [ ]:
train_dataset[i].y

1